## Importations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import scipy.stats as sst
import string
from datetime import datetime, date

import xlrd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import (
    coordinate_from_string,
    column_index_from_string,
    get_column_letter
    )

## Preliminary functions

In [ ]:
def col2num(col):
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num

#### Data extraction
The following function takes as an input a .csv file from the INSTRON machine and return it as an array. (Time, displacement, load )

In [2]:
def extract_data_csv(file):#Extract data from the csv file of the INSTRON
    data=[]
    with open(file) as csvfile:
        datacsv = csv.reader(csvfile)
        for i in range(5):
            next(datacsv)
        for row in datacsv:
            temp_row=[]
            for i in row:
                temp_row.append(float(i))
            data.append(temp_row)
    return(data)

#### Conversion to stress and strain
This function takes as a input the previous array, the cross-sectionnal area of the sample and its initial length to compute the stress and the strain.

In [4]:
def Load_extension(data,area,initial_length):#Converts the load, displacement, Diameter and initial length into stress and strain
    stress=[]
    strain=[]
    for i in range(len(data)):
        stress.append(data[i][2]/(area/4))# Stress = Load/Cross_sectionnal_area
        strain.append(data[i][1]/(initial_length)) # Strain = Elongation/Initial_length
    return stress, strain

#### Selection of the range of strain used to make the linear fit
This function takes as a input the stress/strain array, a superior and inferior strain threshold. It returns the portion of the curve that lies between the interval.

In [ ]:
def fit_portion(strain,stress,thresh_inf,thresh_sup):
    stress_portion=[]
    strain_portion=[]
    for i in range(len(strain)):
        if strain[i]>thresh_inf and strain[i]<thresh_sup:
            stress_portion.append(stress[i])
            strain_portion.append(strain[i])
    return stress_portion,strain_portion

#### Computation of the stiffness
This function takes as a input the stress/strain array and a threshold couple. It returns the slope of a linear fit within the interval.

In [5]:
def plot_data(thresh_inf,thresh_sup,stress,strain):
    stress_portion,strain_portion=fit_portion(strain,stress,thresh_inf,thresh_sup)
    return sst.linregress(strain_portion,stress_portion)